In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Roraima - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_RR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Desemprego,Roraima - value,Roraima - Consumo de Cimento (t)
0,2003-1,0.734916,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,6.128460e+06,332580.202821,12.101540,5.339136e+06,8.263084,0.294583,6.920
1,2003-2,0.734957,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,6.136622e+06,332914.755543,12.103485,5.342057e+06,8.257123,0.296505,7.476
2,2003-3,0.734998,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,6.144784e+06,333249.308265,12.105429,5.344978e+06,8.251161,0.298178,8.659
3,2003-4,0.735038,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,6.152947e+06,333583.860987,12.107373,5.347899e+06,8.245199,0.299598,7.259
4,2003-5,0.735079,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,6.161109e+06,333918.413709,12.109318,5.350821e+06,8.239238,0.300965,6.786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.581560,9.133
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.579849,10.622
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.577718,10.300
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.575950,10.258


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Desemprego,Roraima - value
0,-0.346657,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.650047,-2.537523,-1.382546,-1.820180,-0.695651,-0.837394
1,-0.316945,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.633241,-2.453668,-1.340990,-1.796532,-0.700839,-0.805077
2,-0.287234,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.616435,-2.369814,-1.299433,-1.772885,-0.706028,-0.776961
3,-0.257523,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.599629,-2.285960,-1.257877,-1.749238,-0.711217,-0.753079
4,-0.227812,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.582822,-2.202105,-1.216320,-1.725591,-0.716406,-0.730102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-3.166795,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.253468,-0.612057,-1.780820,1.251634,1.267469,1.568039
188,-3.318917,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.245588,-0.616438,-1.802446,1.242511,1.252097,1.600280
189,-3.471040,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.237708,-0.620818,-1.824072,1.233388,1.236726,1.635777
190,-3.623162,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.229828,-0.625199,-1.845698,1.224265,1.221354,1.674397


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      5.753
1      5.006
2      5.470
3      1.513
4      3.706
       ...  
235      NaN
236      NaN
237      NaN
238      NaN
239      NaN
Name: Roraima - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Desemprego,Roraima - value
0,-0.346657,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.650047,-2.537523,-1.382546,-1.820180,-0.695651,-0.837394
1,-0.316945,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.633241,-2.453668,-1.340990,-1.796532,-0.700839,-0.805077
2,-0.287234,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.616435,-2.369814,-1.299433,-1.772885,-0.706028,-0.776961
3,-0.257523,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.599629,-2.285960,-1.257877,-1.749238,-0.711217,-0.753079
4,-0.227812,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.582822,-2.202105,-1.216320,-1.725591,-0.716406,-0.730102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.256163,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.210022,-0.560021,-0.995787,1.154680,1.464445,1.181073
158,-0.349321,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.218026,-0.570604,-1.003856,1.167190,1.447722,1.186391
159,-0.442479,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.226031,-0.581187,-1.011926,1.179699,1.430999,1.191228
160,-0.535637,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.234036,-0.591770,-1.019995,1.192209,1.414277,1.195247


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       5.753
1       5.006
2       5.470
3       1.513
4       3.706
        ...  
157     6.557
158     8.184
159     8.092
160     7.806
161    10.831
Name: Roraima - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 14)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Desemprego,Roraima - value
126,0.330327,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.751030,1.060935,0.050149,0.761717,0.763630,0.559901
127,0.319821,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.769813,0.950350,0.001472,0.775053,0.820001,0.589214
128,0.309316,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.788597,0.839766,-0.047205,0.788390,0.876373,0.618740
129,0.298810,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.807381,0.729182,-0.095883,0.801726,0.932744,0.648481
130,0.288304,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.826164,0.618597,-0.144560,0.815063,0.989115,0.678511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-3.166795,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.253468,-0.612057,-1.780820,1.251634,1.267469,1.568039
188,-3.318917,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.245588,-0.616438,-1.802446,1.242511,1.252097,1.600280
189,-3.471040,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.237708,-0.620818,-1.824072,1.233388,1.236726,1.635777
190,-3.623162,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.229828,-0.625199,-1.845698,1.224265,1.221354,1.674397


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 14)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, 
                              reshaped_target, 
                              reshaped_test, 
                              reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1036214591, 3205447339, 2245284582, 2063696993, 1200103625, 2148159348, 3532864038, 2275320748, 1695350006, 1685516958, 2316510745, 1414372091, 4153763365, 968377229, 1185863045, 188294346, 302540512, 3505703502, 2216689680, 2714516494, 3877017406, 2759988053, 2165612861, 1887207112, 495274164, 3173651006, 1619993491, 2338016140, 2252547772, 3250533334, 2780506993, 2194521813, 3663306616, 821663841, 2548227478, 1628086852, 746505340, 295037212, 1002894221, 3530813461, 504955242, 2100339988, 963190193, 3518207889, 3916096279, 2691786338, 4119333787, 2363972087, 577319484, 3937925379]


Step: 0 ___________________________________________
val_loss: 0.7030625343322754
winner_seed: 1036214591


Step: 1 ___________________________________________
val_loss: 1.972532868385315


Step: 2 ___________________________________________
val_loss: 0.808915376663208


Step: 3 ___________________________________________
val_loss: 0.6966611742973328
winner_seed: 2063696993


Step: 4 _____________________

In [35]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 114ms/step - loss: 62.4178 - val_loss: 47.1920
Epoch 2/10000
4/4 [==============================] - 0s 7ms/step - loss: 27.6416 - val_loss: 7.1259
Epoch 3/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.0305 - val_loss: 3.8100
Epoch 4/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.7507 - val_loss: 4.1942
Epoch 5/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.6819 - val_loss: 2.0715
Epoch 6/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.7505 - val_loss: 2.0714
Epoch 7/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.2095 - val_loss: 1.9597
Epoch 8/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.7231 - val_loss: 1.8210
Epoch 9/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.4397 - val_loss: 2.0126
Epoch 10/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.5246 - val_loss: 1.9886
Epoc

Epoch 83/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2782 - val_loss: 2.2057
Epoch 84/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2092 - val_loss: 2.5074
Epoch 85/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2230 - val_loss: 2.6428
Epoch 86/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.1886 - val_loss: 2.2048
Epoch 87/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.3606 - val_loss: 2.5344
Epoch 88/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.1705 - val_loss: 2.2885
Epoch 89/10000
4/4 [==============================] - 0s 9ms/step - loss: 1.1532 - val_loss: 2.4279
Epoch 90/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.1888 - val_loss: 2.6726
Epoch 91/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.3772 - val_loss: 2.7675
Epoch 92/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.3202 - val_loss: 2.3878


4/4 [==============================] - 0s 8ms/step - loss: 1.1973 - val_loss: 2.5460
Epoch 246/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.1598 - val_loss: 2.5016
Epoch 247/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.2031 - val_loss: 2.3050
Epoch 248/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.1597 - val_loss: 2.3684
Epoch 249/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2086 - val_loss: 2.4344
Epoch 250/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.2856 - val_loss: 2.0824
Epoch 251/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.1516 - val_loss: 2.3014
Epoch 252/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.2180 - val_loss: 2.4189
Epoch 253/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.1450 - val_loss: 2.2551
Epoch 254/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.2806 - val_loss: 2.5838
Epoch 

4/4 [==============================] - 0s 8ms/step - loss: 1.2025 - val_loss: 2.1270
Epoch 408/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.1808 - val_loss: 2.2092
Epoch 409/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.1412 - val_loss: 2.1950
Epoch 410/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.1936 - val_loss: 2.2779
Epoch 411/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.1312 - val_loss: 2.2387
Epoch 412/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.1496 - val_loss: 2.3697
Epoch 413/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.2096 - val_loss: 2.1406
Epoch 414/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.1914 - val_loss: 2.3527
Epoch 415/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.1456 - val_loss: 2.0909
Epoch 416/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.1700 - val_loss: 2.1944
Epoch 

4/4 [==============================] - 0s 7ms/step - loss: 1.2000 - val_loss: 2.2521
Epoch 570/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.1910 - val_loss: 2.2047
Epoch 571/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.1721 - val_loss: 2.2374
Epoch 572/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.1312 - val_loss: 2.2474
Epoch 573/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.1182 - val_loss: 2.2226
Epoch 574/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.1475 - val_loss: 2.1792
Epoch 575/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2406 - val_loss: 2.3891
Epoch 576/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.1487 - val_loss: 2.1908
Epoch 577/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.1292 - val_loss: 2.1722
Epoch 578/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.1964 - val_loss: 2.2397
Epoch 

...

4/4 [==============================] - 0s 13ms/step - loss: 0.3890 - val_loss: 0.7448
Epoch 5927/10000
4/4 [==============================] - 0s 12ms/step - loss: 0.4144 - val_loss: 0.7706
Epoch 5928/10000
4/4 [==============================] - 0s 13ms/step - loss: 0.3884 - val_loss: 0.7444
Epoch 5929/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.4181 - val_loss: 0.7740
Epoch 5930/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.4253 - val_loss: 0.7100
Epoch 5931/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.4579 - val_loss: 0.6992
Epoch 5932/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.4325 - val_loss: 0.6877
Epoch 5933/10000
4/4 [==============================] - 0s 13ms/step - loss: 0.4153 - val_loss: 0.7623
Epoch 5934/10000
4/4 [==============================] - 0s 13ms/step - loss: 0.4507 - val_loss: 0.7387
Epoch 5935/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.4442 - val_

4/4 [==============================] - 0s 13ms/step - loss: 0.4088 - val_loss: 0.7940
Epoch 6085/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.3661 - val_loss: 0.8541
Epoch 6086/10000
4/4 [==============================] - 0s 13ms/step - loss: 0.4178 - val_loss: 0.7583
Epoch 6087/10000
4/4 [==============================] - 0s 15ms/step - loss: 0.3910 - val_loss: 0.8407
Epoch 6088/10000
4/4 [==============================] - 0s 13ms/step - loss: 0.4094 - val_loss: 0.8408
Epoch 6089/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.4360 - val_loss: 0.8147
Epoch 6090/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.4755 - val_loss: 0.8795
Epoch 6091/10000
4/4 [==============================] - 0s 13ms/step - loss: 0.4262 - val_loss: 0.7465
Epoch 6092/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.3708 - val_loss: 0.7164
Epoch 6093/10000
4/4 [==============================] - 0s 12ms/step - loss: 0.3889 - val_

4/4 [==============================] - 0s 12ms/step - loss: 0.3909 - val_loss: 0.8411
Epoch 6243/10000
4/4 [==============================] - 0s 13ms/step - loss: 0.4388 - val_loss: 0.7376
Epoch 6244/10000
4/4 [==============================] - 0s 20ms/step - loss: 0.3702 - val_loss: 0.8127
Epoch 6245/10000
4/4 [==============================] - 0s 13ms/step - loss: 0.3887 - val_loss: 0.8462
Epoch 6246/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.3858 - val_loss: 0.8286
Epoch 6247/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.3895 - val_loss: 0.7459
Epoch 6248/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.4096 - val_loss: 0.8130
Epoch 6249/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.3903 - val_loss: 0.7320
Epoch 6250/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.3906 - val_loss: 0.7424
Epoch 6251/10000
4/4 [==============================] - 0s 13ms/step - loss: 0.4155 - val_

4/4 [==============================] - 0s 13ms/step - loss: 0.4127 - val_loss: 0.8070
Epoch 6401/10000
4/4 [==============================] - 0s 13ms/step - loss: 0.3626 - val_loss: 0.7555
Epoch 6402/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.4211 - val_loss: 0.7823
Epoch 6403/10000
4/4 [==============================] - 0s 12ms/step - loss: 0.3821 - val_loss: 0.8364
Epoch 6404/10000
4/4 [==============================] - 0s 12ms/step - loss: 0.4762 - val_loss: 0.8303
Epoch 6405/10000
4/4 [==============================] - 0s 12ms/step - loss: 0.3714 - val_loss: 0.7978
Epoch 6406/10000
4/4 [==============================] - 0s 13ms/step - loss: 0.3532 - val_loss: 0.7100
Epoch 6407/10000
4/4 [==============================] - 0s 13ms/step - loss: 0.3525 - val_loss: 0.7061
Epoch 6408/10000
4/4 [==============================] - 0s 13ms/step - loss: 0.3549 - val_loss: 0.8642
Epoch 6409/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.3553 - val_

4/4 [==============================] - 0s 11ms/step - loss: 0.3878 - val_loss: 0.9101
Epoch 6559/10000
4/4 [==============================] - 0s 13ms/step - loss: 0.3581 - val_loss: 0.8061
Epoch 6560/10000
4/4 [==============================] - 0s 12ms/step - loss: 0.4221 - val_loss: 0.7853
Epoch 6561/10000
4/4 [==============================] - 0s 13ms/step - loss: 0.3858 - val_loss: 0.7932
Epoch 6562/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.4287 - val_loss: 0.7665
Epoch 6563/10000
4/4 [==============================] - 0s 15ms/step - loss: 0.3598 - val_loss: 0.8472
Epoch 6564/10000
4/4 [==============================] - 0s 13ms/step - loss: 0.4164 - val_loss: 0.7862
Epoch 6565/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.4076 - val_loss: 0.7405
Epoch 6566/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.3750 - val_loss: 0.8410
Epoch 6567/10000
4/4 [==============================] - 0s 14ms/step - loss: 0.3677 - val_

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 16ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,8.10136,8.103862,8.106921,8.105438,8.099143,8.09197,8.082566,8.071795,8.062688,8.057545,8.054873,8.055198,8.059026,8.064168,8.072199,8.081545,8.09037,8.099209,8.110824,8.118307,8.130765,8.142136,8.147781,8.157026,8.162448,8.171336,8.173038,8.179313,8.184808,8.19311
Target,7.703,8.956,8.626,6.735,9.161,8.634,7.115,2.781,7.49,7.043,4.877,7.031,5.988,7.275,9.468,7.97,7.866,8.108,9.908,10.723,10.625,10.593,10.681,7.887,9.751,9.654,9.422,11.16,10.491,10.225
Error,0.39836,0.852139,0.519079,1.370438,1.061857,0.542029,0.967566,5.290794,0.572688,1.014544,3.177874,1.024198,2.071026,0.789168,1.395802,0.111545,0.22437,0.008791,1.797176,2.604692,2.494235,2.450865,2.533218,0.270026,1.588552,1.482664,1.248962,2.980687,2.306192,2.03189


In [38]:
display(mae)
display(mape)

1.5060478

0.21957748

In [41]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [42]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 17ms/step
Ano-0: |Prediction[[96.993355]] - Target[86.152]| =  Error: [[10.841354]]; MAPE:[[0.12583984]]
1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[97.27336]] - Target[107.092]| =  Error: [[9.818642]]; MAPE:[[0.09168417]]
1/1 [==============================] - 0s 17ms/step
Ano-5: |Prediction[[49.064053]] - Target[60.703]| =  Error: [[11.638947]]; MAPE:[[0.19173594]]


[array([[10.841354]], dtype=float32),
 array([[9.818642]], dtype=float32),
 array([[11.638947]], dtype=float32)]

10.7663145

0.13642